In [2]:
# Access astronomical databases
from pyvo import registry  # version >=1.6

# Moc and HEALPix tools
from mocpy import MOC

# Coordinates manipulation
from astropy.coordinates import SkyCoord

# For plots
import matplotlib.pyplot as plt


# Welcome to VizieR example workflow

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

**Notes:** 

It is a generic notebook, highlighting what can be done once you chose a catalog. This workflow is suggested by [CDS](https://cdsweb.unistra.fr/) (Strasbourg Astronomical Data Center, house of [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR)).

The notebook exploits [pyVO](https://pyvo.readthedocs.io/en/latest/), an advanced library  of The [Virtual Observatory](https://ivoa.net/).

[Astroquery](https://astroquery.readthedocs.io/en/latest/vizier/vizier.html) (not used here) is a well-documented, user-friendly alternative.

--------------------------------------------------------

## 1. Setup

This example notebook has the following dependencies: 

**Required**
- pyvo : this library facilitates the access to the Virtual Observatory (VO) resources. VizieR is part of the VO.
This notebook needs version >=1.4.1
**Optional, for visualization**
- ipyaladin : this is the Aladin-lite sky viewer, bundled as a jupyter widget. It allows to plot catalogs and multi-order coverages (MOC)
- matplotlib : an other option to see catalog points and MOCs

## 2. Metadata exploration with the Virtual Observatory registry

This part uses [pyvo](https://pyvo.readthedocs.io/en) to connect to the VO registry.

In [3]:
# the catalogue name in VizieR
CATALOGUE = "J/A+A/686/A42"

We first retrieve the catalogue information.

In [4]:
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

In [5]:
# We can print metadata information about the catalogue
voresource.describe(verbose=True)

Improving the open cluster census. III.
Short Name: J/A+A/686/A42
IVOA Identifier: ivo://cds.vizier/j/a+a/686/a42
Access modes: conesearch, tap#aux, web
- tap#aux: https://tapvizier.cds.unistra.fr/TAPVizieR/tap
- webpage: https://vizier.cds.unistra.fr/viz-bin/VizieR-2?-source=J/A+A/686/A42
- conesearch: https://vizier.cds.unistra.fr/viz-
 bin/conesearch/J/A+A/686/A42/clusters?, description: Cone search capability
 for table J/A+A/686/A42/clusters (Main catalogue (table 1))
- conesearch: https://vizier.cds.unistra.fr/viz-
 bin/conesearch/J/A+A/686/A42/members?, description: Cone search capability
 for table J/A+A/686/A42/members (Member stars of clusters (table 2))

The census of open clusters has exploded in size thanks to data from the Gaia
satellite. However, it is likely that many of these reported clusters are not
gravitationally bound, making the open cluster census impractical for many
scientific applications. We aim to test different physically motivated methods
for distinguishi

We can also inspect in details the `resource` object and access the attributes not provided by the describe method. See for example, the first author of a resource: 

In [6]:
voresource.creators[0]

'Hunt E.L.'

## 3. Access the tabular data of this catalog

We can have a look at the tables available in the catalogue.

In [7]:
tables = voresource.get_tables()
print(f"In this catalogue, we have {len(tables)} tables.")
for table_name, table in tables.items():
    print(f"{table_name}: {table.description}")

In this catalogue, we have 3 tables.
J/A+A/686/A42/clusters: Main catalogue (table 1)
J/A+A/686/A42/members: Member stars of clusters (table 2)
J/A+A/686/A42/crossma: All (non-)xmatched clusters


In [8]:
# We can also extract the tables names for later use
tables_names = list(tables.keys())
tables_names

['J/A+A/686/A42/clusters', 'J/A+A/686/A42/members', 'J/A+A/686/A42/crossma']

The actual data can then be accessed using any of the ``access_modes`` of the voresource.

In [9]:
voresource.access_modes()

{'conesearch', 'tap#aux', 'web'}

The web access is found by following the ``reference_url``

In [10]:
voresource.reference_url

'https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/686/A42'

<!-- section position -->

We will explore the other access modes here.

### 3.2. Execute a cone search query

We use the Simple Cone Search (<a href='https://www.ivoa.net/documents/latest/ConeSearch.html'>SCS</a>) protocol of the virtual observatory (only available for tables with positions).

Finding the conesearch service that you want to use (there is usually one per table):

Let's use the first one for this example. 

In [11]:
# we get the conesearch  service associated to the first table
conesearch_interface = voresource.get_interface(service_type='conesearch', 
                                                keyword='J/A+A/686/A42/clusters',
                                                lax=True)
# if you get a TypeError about an unexpected keyword, check that you installed pyvo>=1.6
conesearch_service = conesearch_interface.to_service()

We adapt the radius and position of the center to our needs:

In [12]:
conesearch_radius = 1 / 60.0  # in degrees
conesearch_center = (0.054500, 61.065191)

In [13]:
conesearch_records = conesearch_service.search(
    pos=conesearch_center,
    sr=conesearch_radius,
)
conesearch_records

<DALResultsTable length=1>
   _r    recno   Name    ID  ... XmatchType  Note   _RA.icrs     _DE.icrs  
                             ...                      deg          deg     
float64  int32  str20  int16 ...   str16    str33   float64      float64   
-------- ----- ------- ----- ... ---------- ----- ------------ ------------
0.000000  2530 HSC_932  2572 ...        m:1         0.05451013  61.06519563

In [20]:
conesearch_records.to_table()

_r,recno,Name,ID,AllNames,Type,CST,N,CSTt,Nt,RA_ICRS,DE_ICRS,GLON,GLAT,r50,rc,rt,rtot,r50pc,rcpc,rtpc,rtotpc,pmRA,s_pmRA,e_pmRA,pmDE,s_pmDE,e_pmDE,Plx,s_Plx,e_Plx,dist16,dist50,dist84,Ndist,globalPlx,X,Y,Z,RV,s_RV,e_RV,n_RV,CMDCl2.5,CMDCl16,CMDCl50,CMDCl84,CMDCl97.5,CMDClHuman,logAge16,logAge50,logAge84,AV16,AV50,AV84,diffAV16,diffAV50,diffAV84,MOD16,MOD50,MOD84,r50J,rJ,r50Jpc,rJpc,probJ,NJ,MassJ,e_MassJ,MassTot,e_MassTot,minClSize,isMerged,isGMMMemb,NXmatches,XmatchType,Note,_RA.icrs,_DE.icrs
,,,,,,,,,,deg,deg,deg,deg,deg,deg,deg,deg,pc,pc,pc,pc,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas,mas,mas,pc,pc,pc,,,pc,pc,pc,km / s,km / s,km / s,,,,,,,,log(yr),log(yr),log(yr),mag,mag,mag,mag,mag,mag,mag,mag,mag,deg,deg,pc,pc,,,solMass,solMass,solMass,solMass,,,,,,,deg,deg
float64,int32,str20,int16,str253,str1,float64,int32,float64,int32,float64,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,uint8,float64,float64,float64,float64,float64,float64,int16,float32,float32,float64,float64,float64,str3,float64,float64,float64,float32,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,int16,float64,float64,float64,float64,int16,uint8,uint8,int16,str16,str33,float64,float64
0.000000,2530,HSC_932,2572,HSC_932,m,6.73517941,32,6.76520464,30,0.05450046,61.06519101,116.77496538,-1.19,0.51023632,1.05097155,1.05097155,1.17429489,6.49606683,13.38157737,13.38157737,14.95221806,-1.05273012,0.15582492,0.02844961,-1.03844479,0.10486757,0.01914611,1.33462655,0.03861715,0.00705049,727.33036518,729.44116547,731.56425302,28,0,-8450.54346567,651.09169626,6.45752331,-29.49988762,9.37165116,2.70536266,12,0.00654,0.00654,0.24057178,0.41467139,0.82319534,,7.69078588,7.95387244,8.15225124,0.907,1.10204494,1.29729390,0.756,1.21984959,1.62297952,9.12364388,9.28608847,9.42377186,0.10802203,0.34402941,1.37524625,4.37994266,0.2,10,25.24378745,4.00032460,78.04860240,11.80847762,20,0,0,1,m:1,,0.05451013,61.06519563


## 4. Get the catalogue coverage

VizieR also provides the coverage of each catalogue with Multiple Order Coverage (<a href='https://ivoa.net/documents/MOC/'>MOC</a>) at order 10 -- meaning with a 0.001 rad spacial resolution.

In [ ]:
# retrieve the MOC
catalogue_coverage = MOC.from_vizier_table(CATALOGUE)
catalogue_coverage.display_preview()

The quick preview is the whole sky in equatorial coordinates. Red represent zones where the catalog has data, black areas are empty zones.

We can also plot the coverage with ``matplotlib``.

In [ ]:
fig = plt.figure(figsize=(5, 5))
wcs = catalogue_coverage.wcs(fig)
ax = fig.add_subplot(projection=wcs)
catalogue_coverage.fill(ax=ax, wcs=wcs, alpha=0.5, color="blue")